# Employee Turn Over

Problem Statement :

We would like to predict Employee Turn over based on employee application activity.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
PATH_DATA='../../data/raw/'
PATH_DATA_INTERIM='../../data/iter2/interim/'

In [3]:

def q50(x):
    return x.quantile(0.5)

def q75(x):
    return x.quantile(0.75)

def get_unique_id(df) :
    return  df['employee'].astype(str)+'_'+df['companyAlias']

## Get The data

preview all table

In [4]:
data_churn=pd.read_csv(PATH_DATA+'churn.csv')
data_comment=pd.read_csv(PATH_DATA+'commentInteractions.csv')
data_votes=pd.read_csv(PATH_DATA+'votes.csv')
data_comment_cln0=pd.read_csv(PATH_DATA+'comments_clean_anonimized.csv')

In [14]:
## Happy today?
data_votes.sample(10)

,employee,companyAlias,voteDate,vote
185277,69,57ac8b23be7fe30003e656d0,Sun Mar 05 01:00:00 CET 2017,1
174410,77,53a2dd43e4b01cc02f1e9011,Mon Mar 21 01:00:00 CET 2016,3
6573,114,56aec740f1ef260003e307d6,Tue Jul 12 02:00:00 CEST 2016,1
78540,205,54e52607e4b01191dc064966,Thu Mar 03 01:00:00 CET 2016,3
175161,144,53a2dd43e4b01cc02f1e9011,Fri Apr 08 02:00:00 CEST 2016,3
44424,62,5809cc9eff2ea40003fda44d,Thu Feb 09 01:00:00 CET 2017,2
110859,3,57dd2d6a4018d9000339ca43,Sun Nov 06 01:00:00 CET 2016,3
144731,8,56fd2b64f41c670003f643c8,Fri Nov 25 01:00:00 CET 2016,3
72896,453,54e52607e4b01191dc064966,Wed Nov 18 01:00:00 CET 2015,3
126169,223,5641f96713664c000332c8cd,Wed Aug 31 02:00:00 CEST 2016,3


In [6]:
data_votes.shape

(221232, 4)

In [7]:
data_comment.head()

,employee,companyAlias,liked,disliked,commentId
0,307,56aec740f1ef260003e307d6,True,False,58d018d7e010990004e38070
1,36,56aec740f1ef260003e307d6,True,False,58d018d7e010990004e38070
2,276,56aec740f1ef260003e307d6,True,False,58d018d7e010990004e38070
3,24,56aec740f1ef260003e307d6,True,False,58d018d7e010990004e38070
4,382,56aec740f1ef260003e307d6,True,False,58d0179ae010990004e3806d


In [8]:
data_comment.shape

(336960, 5)

In [9]:
data_comment_cln0.head()

,employee,companyAlias,commentId,txt,likes,dislikes,commentDate
0,307,56aec740f1ef260003e307d6,58d018d7e010990004e38070,**********************************************...,4.0,0.0,Mon Mar 20 19:00:17 CET 2017
1,382,56aec740f1ef260003e307d6,58d0179ae010990004e3806d,*****************************,1.0,2.0,Mon Mar 20 18:55:16 CET 2017
2,172,56aec740f1ef260003e307d6,58cff8cde010990004e37f6a,***************************,3.0,0.0,Mon Mar 20 16:44:02 CET 2017
3,135,56aec740f1ef260003e307d6,58cfefeee010990004e37f60,***************************,1.0,1.0,Mon Mar 20 16:06:08 CET 2017
4,225,56aec740f1ef260003e307d6,58cfd9b4e010990004e37f52,*********************************,3.0,2.0,Mon Mar 20 14:30:50 CET 2017


In [18]:
len(data_comment_cln0.commentId),len(data_comment_cln0.commentId.unique())

(82756, 41399)

In [19]:
data_comment_cln0.shape

(82756, 7)

In [20]:
data_churn.head()

,employee,companyAlias,numVotes,lastParticipationDate,stillExists
0,512,56aec740f1ef260003e307d6,4,Thu Feb 23 12:48:04 CET 2017,True
1,-2,56aec740f1ef260003e307d6,0,Wed Jan 18 14:00:55 CET 2017,False
2,2,56aec740f1ef260003e307d6,72,Fri Mar 17 01:00:00 CET 2017,True
3,487,56aec740f1ef260003e307d6,14,Sat Nov 19 15:02:14 CET 2016,False
4,3,56aec740f1ef260003e307d6,22,Thu Feb 16 01:00:00 CET 2017,True


In [21]:
data_churn.shape

(4847, 5)

## Formulate the solutions

    Each table has different level. Company level, Employee level, Comment level, or Votes level. 
    Since our target to create model that predict Employee turn over, then we must create features/predictor on Employee level.
    Also, we must have employee that used for training and employee for testing.
    
    Lets generate employee_target.
    

In [22]:
## One of data with Employee level information is data_churn which also contains the target of our model.
data_churn.head()

,employee,companyAlias,numVotes,lastParticipationDate,stillExists
0,512,56aec740f1ef260003e307d6,4,Thu Feb 23 12:48:04 CET 2017,True
1,-2,56aec740f1ef260003e307d6,0,Wed Jan 18 14:00:55 CET 2017,False
2,2,56aec740f1ef260003e307d6,72,Fri Mar 17 01:00:00 CET 2017,True
3,487,56aec740f1ef260003e307d6,14,Sat Nov 19 15:02:14 CET 2016,False
4,3,56aec740f1ef260003e307d6,22,Thu Feb 16 01:00:00 CET 2017,True


In [23]:
## Employee Id is not unique since different company may have same employee id
len(data_churn.employee.unique())==len(data_churn)

False

In [24]:
data_churn['unique_employee_ids'] = get_unique_id(data_churn)

In [25]:
len(data_churn.unique_employee_ids.unique())==len(data_churn) ## --> still not unique

False

In [26]:
len(data_churn.unique_employee_ids.unique()),len(data_churn)

(4806, 4847)

In [28]:
## for simplicity --> remove the double
employee_data_cnt = data_churn.groupby('unique_employee_ids').size().sort_values()
employee_not_unique = employee_data_cnt[employee_data_cnt>1].index.tolist()
print(len(employee_not_unique))

data_churn2 = data_churn[~(data_churn.unique_employee_ids.isin(employee_not_unique))]

37


In [29]:
len(data_churn2.unique_employee_ids.unique())==len(data_churn2) ## --> unique already.

True

In [30]:
df_employee_target = data_churn2[['unique_employee_ids','stillExists','companyAlias']]
df_employee_target.head()

,unique_employee_ids,stillExists,companyAlias
0,512_56aec740f1ef260003e307d6,True,56aec740f1ef260003e307d6
1,-2_56aec740f1ef260003e307d6,False,56aec740f1ef260003e307d6
2,2_56aec740f1ef260003e307d6,True,56aec740f1ef260003e307d6
4,3_56aec740f1ef260003e307d6,True,56aec740f1ef260003e307d6
5,-4_56aec740f1ef260003e307d6,False,56aec740f1ef260003e307d6


In [31]:
train_size = int(len(df_employee_target)*0.7) ##--> train size
print(train_size)

3338


In [33]:
## Get test and generate flag
employee_train = df_employee_target.sample(train_size,random_state = 103)['unique_employee_ids']
df_employee_target['is_train'] = df_employee_target.unique_employee_ids.isin(employee_train).astype(int)

In [34]:
df_employee_target.head()

,unique_employee_ids,stillExists,companyAlias,is_train
0,512_56aec740f1ef260003e307d6,True,56aec740f1ef260003e307d6,0
1,-2_56aec740f1ef260003e307d6,False,56aec740f1ef260003e307d6,1
2,2_56aec740f1ef260003e307d6,True,56aec740f1ef260003e307d6,0
4,3_56aec740f1ef260003e307d6,True,56aec740f1ef260003e307d6,0
5,-4_56aec740f1ef260003e307d6,False,56aec740f1ef260003e307d6,1


## Employee Features 1

    use data votes.
    remember --> Central European Time – CET Time Zone / European Central Time (Standard Time)
    remember --> hours only 1 and 2 somehow
    possible features in employee level :

    1. Avg Votes
    2. p50 Votes
    3. p75 Votes
    4. Cnt Votes
    5. Std Votes
    6. Avg Votes Monday
    7. p50 Votes Monday
    8. p75 Votes Monday
    9. Cnt Votes Monday
    10. Std Votes Monday
    11. Avg Votes Friday
    12. p50 Votes Friday
    13. p75 Votes Friday
    14. Cnt Votes Friday
    15. Std Votes Friday



In [35]:
data_votes.head()

,employee,companyAlias,voteDate,vote
0,31,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,4
1,33,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,4
2,79,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,4
3,94,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,4
4,16,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,2


In [36]:
data_votes['unique_employee_ids'] = get_unique_id(data_votes)
data_votes['voteDate2'] = pd.to_datetime(data_votes['voteDate'])

In [37]:
data_votes['day_name'] = data_votes['voteDate2'].dt.strftime('%a')
data_votes['votes_monday'] = np.where(data_votes['day_name']=='Mon',data_votes['vote'],np.NaN)
data_votes['votes_friday'] = np.where(data_votes['day_name']=='Fri',data_votes['vote'],np.NaN)

In [41]:
data_votes.sample(8)

,employee,companyAlias,voteDate,vote,unique_employee_ids,voteDate2,day_name,votes_monday,votes_friday
171577,387,574c5ade56b6300003009965,Tue Feb 07 01:00:00 CET 2017,2,387_574c5ade56b6300003009965,2017-02-07 01:00:00,Tue,NaN,NaN
116627,299,57dd2d6a4018d9000339ca43,Fri Feb 10 01:00:00 CET 2017,2,299_57dd2d6a4018d9000339ca43,2017-02-10 01:00:00,Fri,NaN,2.0
8195,131,56aec740f1ef260003e307d6,Mon Aug 08 02:00:00 CEST 2016,3,131_56aec740f1ef260003e307d6,2016-08-08 02:00:00,Mon,3.0,NaN
121092,277,5641f96713664c000332c8cd,Sun Jun 12 02:00:00 CEST 2016,4,277_5641f96713664c000332c8cd,2016-06-12 02:00:00,Sun,NaN,NaN
37498,160,57bb2f0b3bae540003a8d453,Thu Jan 19 01:00:00 CET 2017,4,160_57bb2f0b3bae540003a8d453,2017-01-19 01:00:00,Thu,NaN,NaN
83267,1,54e52607e4b01191dc064966,Thu May 26 02:00:00 CEST 2016,3,1_54e52607e4b01191dc064966,2016-05-26 02:00:00,Thu,NaN,NaN
25054,373,56aec740f1ef260003e307d6,Thu Mar 02 01:00:00 CET 2017,1,373_56aec740f1ef260003e307d6,2017-03-02 01:00:00,Thu,NaN,NaN
38494,82,57bb2f0b3bae540003a8d453,Mon Feb 13 01:00:00 CET 2017,3,82_57bb2f0b3bae540003a8d453,2017-02-13 01:00:00,Mon,3.0,NaN


In [42]:
dg1 = data_votes.groupby('unique_employee_ids')

In [43]:
## group metrics
fun1 = ['count', np.mean, np.std, q50, q75]
fun1_dict = {}
col_f1 = ['unique_employee_ids']
for cols in ['vote', 'votes_monday', 'votes_friday']:
    fun1_dict[cols] = fun1
    for met in ['cnt','avg','std','p50','p75']:
        col_f1.append(cols+'_'+met)
        
emp_f1 = dg1.agg(fun1_dict).reset_index()
emp_f1.columns = col_f1

In [45]:
emp_f1.sample(8)

,unique_employee_ids,vote_cnt,vote_avg,vote_std,vote_p50,vote_p75,votes_monday_cnt,votes_monday_avg,votes_monday_std,votes_monday_p50,votes_monday_p75,votes_friday_cnt,votes_friday_avg,votes_friday_std,votes_friday_p50,votes_friday_p75
2564,353_56aec740f1ef260003e307d6,136,3.985294,0.120818,4.0,4.00,21,3.952381,0.218218,4.0,4.0,19,4.000000,0.000000,4.0,4.0
1182,184_56aec740f1ef260003e307d6,133,2.323308,0.973504,3.0,3.00,20,2.200000,0.951453,3.0,3.0,23,2.086957,1.040675,2.0,3.0
3627,670_54e52607e4b01191dc064966,2,3.500000,0.707107,3.5,3.75,1,4.000000,NaN,4.0,4.0,0,NaN,NaN,NaN,NaN
2146,282_57ac8b23be7fe30003e656d0,17,3.941176,0.242536,4.0,4.00,2,4.000000,0.000000,4.0,4.0,1,3.000000,NaN,3.0,3.0
328,122_5370af43e4b0cff95558c12a,38,3.052632,0.733277,3.0,3.00,13,3.153846,0.800641,3.0,4.0,4,3.250000,0.957427,3.5,4.0
41,103_5370af43e4b0cff95558c12a,61,2.573770,0.717844,3.0,3.00,11,2.000000,1.000000,2.0,2.5,12,2.833333,0.577350,3.0,3.0
4219,92_57d956302a040a00036a8905,5,3.000000,0.707107,3.0,3.00,1,2.000000,NaN,2.0,2.0,1,4.000000,NaN,4.0,4.0
941,167_53a2dd43e4b01cc02f1e9011,96,3.541667,0.578868,4.0,4.00,17,3.470588,0.514496,3.0,4.0,16,3.687500,0.478714,4.0,4.0


## Employee Features 2

    use data comment cln.
    Acpt likes --> likes > 0 
    possible features in employee level :

    1. Avg long text
    2. p50 long text
    3. p75 long text
    4. std long text
    5. cnt text
    6. Avg Acpt likes
    7. p50 Acpt likes
    8. p75 Acpt likes
    9. std Acpt likes
    10. cnt Acpt dislikes
    11. Avg Acpt dislikes
    12. p50 Acpt dislikes
    13. p75 Acpt dislikes
    14. std Acpt dislikes
    15. cnt Acpt dislikes
    16. cnt morning comment
    17. cnt night comment
    18. cnt lunch comment
    19. cnt weekend comment
    20. cnt weekday comment


In [47]:
data_comment_cln0[data_comment_cln0.commentDate.isna()]

,employee,companyAlias,commentId,txt,likes,dislikes,commentDate
82755,361,58a728a0e75bda00042a3468,58adb214b8242400048bf8a6,NaN,NaN,NaN,NaN


In [48]:
data_comment_cln1 = data_comment_cln0[~(data_comment_cln0.commentDate.isna())]

In [50]:
data_comment_cln1.sample(10)

,employee,companyAlias,commentId,txt,likes,dislikes,commentDate
65647,156,53a2dd43e4b01cc02f1e9011,57a4bc13ad0a820003411d89,**************************,0.0,0.0,Fri Aug 05 18:17:04 CEST 2016
26600,3,57908a2622881200033b34d7,57f614f444892f00033aa129,**********************************************...,1.0,0.0,Thu Oct 06 11:08:52 CEST 2016
51833,25,56fd2b64f41c670003f643c8,576341da2d9dc00003ba3d89,*******,1.0,0.0,Fri Jun 17 02:18:15 CEST 2016
42059,278,54e52607e4b01191dc064966,54f44080e4b06b74f59afd6a,*************,2.0,0.0,Mon Mar 02 11:50:38 CET 2015
78171,52,57d956302a040a00036a8905,58243ec8936b910004b12f66,************************************,6.0,0.0,Thu Nov 10 10:32:31 CET 2016
57812,36,57d1eb86a22c9d0003dd1f05,586b38da327a0000042e0237,***********************************************,3.0,0.0,Tue Jan 03 06:38:13 CET 2017
39650,301,54e52607e4b01191dc064966,55cb7733e4b033cfa32cccde,**********************************************...,1.0,0.0,Wed Aug 12 18:41:22 CEST 2015
45178,162,5641f96713664c000332c8cd,5881fec56f31e700046fe298,**********************************************...,8.0,6.0,Fri Jan 20 13:11:37 CET 2017
35105,964,54e52607e4b01191dc064966,5887caa424448e000418a006,**********************************************...,20.0,0.0,Tue Jan 24 22:42:48 CET 2017
27502,208,54e52607e4b01191dc064966,587632535222890004d9d364,************,6.0,2.0,Wed Jan 11 14:25:28 CET 2017


In [51]:
data_comment_cln1['unique_employee_ids'] = get_unique_id(data_comment_cln1)
data_comment_cln1['commentDate2'] = pd.to_datetime(data_comment_cln1['commentDate'])
data_comment_cln1['len_txt'] = data_comment_cln1['txt'].str.len()

In [52]:
# data_comment_cln1[data_comment_cln1.len_txt==0]

In [53]:
data_comment_cln1['is_weekend'] = data_comment_cln1['commentDate2'].dt.strftime('%w').isin(['0','6']).astype(int)
data_comment_cln1['hours'] = data_comment_cln1['commentDate2'].dt.strftime('%H').astype(int)

data_comment_cln1['comment_weekend'] = np.where(data_comment_cln1['is_weekend']==1,data_comment_cln1['txt'],np.NaN)
data_comment_cln1['comment_weekday'] = np.where(data_comment_cln1['is_weekend']==0,data_comment_cln1['txt'],np.NaN)
data_comment_cln1['comment_morning'] = np.where((data_comment_cln1['hours']>=5)&
                                                (data_comment_cln1['hours']<10),data_comment_cln1['txt'],np.NaN)
data_comment_cln1['comment_night'] = np.where((data_comment_cln1['hours']>=19)&
                                                (data_comment_cln1['hours']<=23),data_comment_cln1['txt'],np.NaN)
data_comment_cln1['comment_lunch'] = np.where((data_comment_cln1['hours']>=11)&
                                                (data_comment_cln1['hours']<14),data_comment_cln1['txt'],np.NaN)

data_comment_cln1['acpt_likes'] = np.where(data_comment_cln1.likes>0,data_comment_cln1.likes,np.NaN)
data_comment_cln1['acpt_dislikes'] = np.where(data_comment_cln1.dislikes>0,data_comment_cln1.likes,np.NaN)

In [54]:
data_comment_cln1.sample(8)

,employee,companyAlias,commentId,txt,likes,dislikes,commentDate,unique_employee_ids,commentDate2,len_txt,is_weekend,hours,comment_weekend,comment_weekday,comment_morning,comment_night,comment_lunch,acpt_likes,acpt_dislikes
50932,57,56fd2b64f41c670003f643c8,57a22482534043000326c1f2,**********************************************...,4.0,0.0,Wed Aug 03 19:05:22 CEST 2016,57_56fd2b64f41c670003f643c8,2016-08-03 19:05:22,73.0,0,19,NaN,**********************************************...,NaN,**********************************************...,NaN,4.0,NaN
38546,51,54e52607e4b01191dc064966,5659ab4619a7030003678cea,**********************************************...,5.0,0.0,Sat Nov 28 14:25:28 CET 2015,51_54e52607e4b01191dc064966,2015-11-28 14:25:28,77.0,1,14,**********************************************...,NaN,NaN,NaN,NaN,5.0,NaN
53958,6,56e2a905e3b6fe0003e32855,58332f39eafa8e0004becadc,**********************************************...,3.0,6.0,Mon Nov 21 18:29:11 CET 2016,6_56e2a905e3b6fe0003e32855,2016-11-21 18:29:11,151.0,0,18,NaN,**********************************************...,NaN,NaN,NaN,3.0,3.0
38066,262,54e52607e4b01191dc064966,56ad126a63b2d50003f21aab,*******,1.0,1.0,Sat Jan 30 20:43:29 CET 2016,262_54e52607e4b01191dc064966,2016-01-30 20:43:29,7.0,1,20,*******,NaN,NaN,*******,NaN,1.0,1.0
40582,3,54e52607e4b01191dc064966,5576b1e5e4b028b811f7552d,**********************************************...,6.0,0.0,Tue Jun 09 11:29:09 CEST 2015,3_54e52607e4b01191dc064966,2015-06-09 11:29:09,106.0,0,11,NaN,**********************************************...,NaN,NaN,**********************************************...,6.0,NaN
20321,60,581b08041a0ef8000308aef6,586bae1a327a0000042e0261,******************,1.0,0.0,Tue Jan 03 14:02:50 CET 2017,60_581b08041a0ef8000308aef6,2017-01-03 14:02:50,18.0,0,14,NaN,******************,NaN,NaN,NaN,1.0,NaN
69778,235,57ac8b23be7fe30003e656d0,58ad963bb8242400048bf853,***********,0.0,0.0,Wed Feb 22 14:45:08 CET 2017,235_57ac8b23be7fe30003e656d0,2017-02-22 14:45:08,11.0,0,14,NaN,***********,NaN,NaN,NaN,NaN,NaN
45905,87,5641f96713664c000332c8cd,580e567c4c91830003b40a7a,**********************************************...,19.0,7.0,Mon Oct 24 20:42:14 CEST 2016,87_5641f96713664c000332c8cd,2016-10-24 20:42:14,252.0,0,20,NaN,**********************************************...,NaN,**********************************************...,NaN,19.0,19.0


In [55]:
dg2 = data_comment_cln1.groupby('unique_employee_ids')

In [56]:
## group metrics 1
fun2_1 = ['count', np.mean, np.std, q50, q75]
fun2_1_dict = {}
col_f2_1 = ['unique_employee_ids']
for cols in  ['acpt_likes','acpt_dislikes', 'len_txt'] :
    fun2_1_dict[cols] = fun2_1
    for met in ['cnt','avg','std','p50','p75']:
        col_f2_1.append(cols+'_'+met)
        
emp_f2_1 = dg2.agg(fun2_1_dict).reset_index()
emp_f2_1.columns = col_f2_1

In [57]:
## group metrics 1
fun2_2 = ['count']
fun2_2_dict = {}
col_f2_2 = ['unique_employee_ids']
for cols in ['comment_weekend','comment_weekday','comment_morning','comment_night','comment_lunch']:
    fun2_2_dict[cols] = fun2_2
    for met in ['cnt']:
        col_f2_2.append(cols+'_'+met)
        
emp_f2_2 = dg2.agg(fun2_2_dict).reset_index()
emp_f2_2.columns = col_f2_2

In [58]:
emp_f2_1.sample(3)

,unique_employee_ids,acpt_likes_cnt,acpt_likes_avg,acpt_likes_std,acpt_likes_p50,acpt_likes_p75,acpt_dislikes_cnt,acpt_dislikes_avg,acpt_dislikes_std,acpt_dislikes_p50,acpt_dislikes_p75,len_txt_cnt,len_txt_avg,len_txt_std,len_txt_p50,len_txt_p75
3002,839_58a728a0e75bda00042a3468,6,6.333333,4.501851,7.0,10.0,6,6.333333,4.501851,7.0,10.0,6,95.000000,51.154667,64.0,136.75
2836,70_567011c035dce00003a07fa4,10,3.000000,2.309401,3.0,3.0,0,NaN,NaN,NaN,NaN,12,136.833333,162.148548,55.5,234.00
1544,243_57ac8b23be7fe30003e656d0,4,1.000000,0.000000,1.0,1.0,6,0.666667,0.516398,1.0,1.0,14,6.000000,6.702468,4.0,8.75


In [59]:
emp_f2_2.sample(3)

,unique_employee_ids,comment_weekend_cnt,comment_weekday_cnt,comment_morning_cnt,comment_night_cnt,comment_lunch_cnt
2465,49_57908a2622881200033b34d7,0,12,0,0,4
3180,97_57bb2f0b3bae540003a8d453,0,8,0,2,0
1350,210_567011c035dce00003a07fa4,10,154,76,6,34


## Employee Features 3

    use data comment
    possible features in employee level :

    1. cnt gv liked
    2. cnt gv disliked
    3. total reaction


In [63]:
data_comment.sample(10)

,employee,companyAlias,liked,disliked,commentId,unique_employee_ids
250985,271,574c5ade56b6300003009965,True,False,57721fb5e39ab50003faa809,271_574c5ade56b6300003009965
8292,446,56aec740f1ef260003e307d6,True,False,5880650a0e677f0004390aab,446_56aec740f1ef260003e307d6
184660,165,5641f96713664c000332c8cd,False,True,580cc0820be0910003f0f533,165_5641f96713664c000332c8cd
310466,263,57d956302a040a00036a8905,True,False,583680518d0b080004eb2200,263_57d956302a040a00036a8905
265596,147,57ac8b23be7fe30003e656d0,True,False,583dc09c7167c100045f74db,147_57ac8b23be7fe30003e656d0
152668,-179,54e52607e4b01191dc064966,True,False,54eca15be4b0a8ef706f73ff,-179_54e52607e4b01191dc064966
157334,82,57dd2d6a4018d9000339ca43,True,False,5844138397d305000430c350,82_57dd2d6a4018d9000339ca43
197158,15,5641f96713664c000332c8cd,False,True,57b346ee00e1260003c602be,15_5641f96713664c000332c8cd
153462,288,57dd2d6a4018d9000339ca43,True,False,58b7d8977a5e700004eaf8e9,288_57dd2d6a4018d9000339ca43
276598,183,5742d699f839a10003a407d2,True,False,585b851eac0fe00004b24ca3,183_5742d699f839a10003a407d2


In [64]:
data_comment['unique_employee_ids'] = get_unique_id(data_comment)

In [65]:
data_comment['gv_liked'] = np.where(data_comment.liked==True,1,0)
data_comment['gv_disliked'] = np.where(data_comment.disliked==True,1,0)

In [66]:
dg3 = data_comment.groupby('unique_employee_ids')

In [67]:
emp_f3 = dg3.agg({'gv_liked':np.sum, 'gv_disliked':np.sum}).reset_index()
emp_f3.columns = ['unique_employee_ids','gv_liked_cnt','gv_disliked_cnt']
emp_f3['reaction_cnt'] = emp_f3['gv_liked_cnt'] + emp_f3['gv_disliked_cnt']

In [71]:
emp_f3.sample(3)

,unique_employee_ids,gv_liked_cnt,gv_disliked_cnt,reaction_cnt
3291,84_54e52607e4b01191dc064966,8,1,9
1466,212_567011c035dce00003a07fa4,96,51,147
248,-4_57dd2d6a4018d9000339ca43,13,0,13


## Company level features

    collective engagement of all employee on a company may impact employee turn over. Such as :
    
    1. Company Turn over rate
    2. Company Avg Votes
    3. Company p50 Votes
    4. Company p75 Votes
    5. Company Cnt Votes
    6. Company Std Votes
    7. comment per employee (mean)
    8. txt len per employee (mean)
    9. comment per employee (p50)
    10. txt len per employee (p50)
    11. liked per employee (mean)
    12. disliked per employee (mean)
    12. liked per employee (p50)
    13. disliked per employee (p50)
    
    this information is obtained from trained data

In [73]:
## Turn over rate
df_employee_target_tr = df_employee_target[df_employee_target.is_train==1]
comp_f1 = (1-df_employee_target_tr.groupby('companyAlias').stillExists.mean()).reset_index()
comp_f1.columns = ['companyAlias','turn_over_rate']

comp_f1.sample(3)

,companyAlias,turn_over_rate
4,54e52607e4b01191dc064966,0.598881
6,5641f96713664c000332c8cd,0.139073
0,5370af43e4b0cff95558c12a,0.844828


In [74]:
data_votes_tr = data_votes[data_votes.unique_employee_ids.isin(employee_train)]

In [75]:
len(data_votes_tr),len(data_votes)

(156281, 221232)

In [76]:
## Votes
dg3 = data_votes_tr.groupby('companyAlias')

fun4 = ['count', np.mean, np.std, q50, q75]
fun4_dict = {}
col_f4 = ['companyAlias']
for cols in ['vote']:
    fun4_dict[cols] = fun4
    for met in ['cnt','avg','std','p50','p75']:
        col_f4.append(cols+'_company_'+met)
        
comp_f2 = dg3.agg(fun4_dict).reset_index()
comp_f2.columns = col_f4

In [77]:
comp_f2.sample(3)

,companyAlias,vote_company_cnt,vote_company_avg,vote_company_std,vote_company_p50,vote_company_p75
7,56558cfd07a5de00030908fb,18,3.222222,0.878204,3,4
9,56ab28dc1f385d0003454757,117,3.076923,1.138313,3,4
26,57dd2d6a4018d9000339ca43,8224,2.971060,0.887104,3,4


In [78]:
## comment cln1
data_comment_cln1_tr = data_comment_cln1[data_comment_cln1.unique_employee_ids.isin(employee_train)]

dg4_metric = data_comment_cln1_tr.groupby(['companyAlias','unique_employee_ids']).agg({'len_txt':['count',np.sum]}).reset_index()
dg4_metric.columns = ['companyAlias','unique_employee_ids','total_comment','total_comment_len']
comp_f3 = dg4_metric.groupby('companyAlias').mean().reset_index()
comp_f3.columns = ['companyAlias','comment_per_emp_avg','comment_len_per_emp_avg']
comp_f4 = dg4_metric.groupby('companyAlias').quantile(0.5).reset_index()
comp_f4.columns = ['companyAlias','comment_per_emp_p50','comment_len_per_emp_p50']

print(comp_f3.sample(3))
print(comp_f4.sample(3))

                companyAlias  comment_per_emp_avg  comment_len_per_emp_avg
3   54d43612e4b0f6a40755d93e            26.750000               647.000000
23  57d956302a040a00036a8905            21.264368              2465.908046
24  57d979b72a040a00036a8925            19.000000               676.000000
                companyAlias  comment_per_emp_p50  comment_len_per_emp_p50
13  56fd2b64f41c670003f643c8                 19.0                    743.0
33  58b9adfce75bf80004df6536                  7.0                    179.0
14  573f2c4a3517490003ef7710                 10.0                    785.0


In [79]:
## comment reaction
data_comment_tr = data_comment[data_comment.unique_employee_ids.isin(employee_train)]
dg5_metric = data_comment_tr.groupby(['companyAlias','unique_employee_ids']).agg({'gv_liked':[np.sum],'gv_disliked':[np.sum]}).reset_index()
dg5_metric.columns = ['companyAlias','unique_employee_ids','gv_liked_exc_cnt','gv_disliked_exc_cnt']

comp_f5 = dg5_metric.groupby('companyAlias').mean().reset_index()
comp_f5.columns = ['companyAlias','gv_liked_per_emp_avg','gv_disliked_per_emp_avg']
comp_f6 = dg5_metric.groupby('companyAlias').quantile(0.5).reset_index()
comp_f6.columns = ['companyAlias','gv_liked_per_emp_p50','gv_disliked_per_emp_p50']

In [80]:
comp_f6.sample(10)

,companyAlias,gv_liked_per_emp_p50,gv_disliked_per_emp_p50
11,56e2a905e3b6fe0003e32855,32.0,9.0
31,58b471b384db3200044dd1b9,6.0,1.0
23,57d979b72a040a00036a8925,2.0,1.0
18,57ac8b23be7fe30003e656d0,5.0,0.0
8,56ab28dc1f385d0003454757,1.0,1.0
6,56558cfd07a5de00030908fb,1.5,0.0
19,57bb2f0b3bae540003a8d453,28.5,4.5
20,57c4aa7dbb8b5c000396fd3b,11.0,1.0
14,5742d699f839a10003a407d2,114.5,7.0
24,57dd2d6a4018d9000339ca43,18.0,1.0


## Merge Features

In [81]:
empl_features = [emp_f1, emp_f2_1, emp_f2_2, emp_f3]
comp_features = [comp_f1, comp_f2, comp_f3, comp_f4, comp_f5, comp_f6]

# all_df_features = empl_features + comp_features

In [82]:
df_interim = df_employee_target.copy()
for d in empl_features:
    df_interim = pd.merge(df_interim, d, on = 'unique_employee_ids', how = 'left')
                          
for d in comp_features:
    df_interim = pd.merge(df_interim, d, on = 'companyAlias', how = 'left')

In [83]:
df_interim

,unique_employee_ids,stillExists,companyAlias,is_train,vote_cnt,vote_avg,vote_std,vote_p50,vote_p75,votes_monday_cnt,...,vote_company_p50,vote_company_p75,comment_per_emp_avg,comment_len_per_emp_avg,comment_per_emp_p50,comment_len_per_emp_p50,gv_liked_per_emp_avg,gv_disliked_per_emp_avg,gv_liked_per_emp_p50,gv_disliked_per_emp_p50
0,512_56aec740f1ef260003e307d6,True,56aec740f1ef260003e307d6,0,4.0,2.500000,1.290994,2.5,3.25,1.0,...,4,4,40.497238,3133.790055,24.0,1522.0,105.754902,30.171569,35.5,4.0
1,-2_56aec740f1ef260003e307d6,False,56aec740f1ef260003e307d6,1,NaN,NaN,NaN,NaN,NaN,NaN,...,4,4,40.497238,3133.790055,24.0,1522.0,105.754902,30.171569,35.5,4.0
2,2_56aec740f1ef260003e307d6,True,56aec740f1ef260003e307d6,0,72.0,2.250000,1.031203,2.0,3.00,13.0,...,4,4,40.497238,3133.790055,24.0,1522.0,105.754902,30.171569,35.5,4.0
3,3_56aec740f1ef260003e307d6,True,56aec740f1ef260003e307d6,0,22.0,3.454545,0.738549,4.0,4.00,2.0,...,4,4,40.497238,3133.790055,24.0,1522.0,105.754902,30.171569,35.5,4.0
4,-4_56aec740f1ef260003e307d6,False,56aec740f1ef260003e307d6,1,NaN,NaN,NaN,NaN,NaN,NaN,...,4,4,40.497238,3133.790055,24.0,1522.0,105.754902,30.171569,35.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4764,857_58a728a0e75bda00042a3468,True,58a728a0e75bda00042a3468,1,1.0,3.000000,NaN,3.0,3.00,0.0,...,3,3,9.088889,931.844444,6.5,360.0,35.469027,10.668142,19.5,4.0
4765,858_58a728a0e75bda00042a3468,True,58a728a0e75bda00042a3468,0,1.0,3.000000,NaN,3.0,3.00,0.0,...,3,3,9.088889,931.844444,6.5,360.0,35.469027,10.668142,19.5,4.0
4766,859_58a728a0e75bda00042a3468,True,58a728a0e75bda00042a3468,1,1.0,4.000000,NaN,4.0,4.00,0.0,...,3,3,9.088889,931.844444,6.5,360.0,35.469027,10.668142,19.5,4.0
4767,17_573a0671b5ec330003add34a,True,573a0671b5ec330003add34a,1,7.0,3.571429,0.534522,4.0,4.00,2.0,...,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
df_interim.to_csv(PATH_DATA_INTERIM+'iter2_interim.csv',index = False)